# Install necessary packages

In [25]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Part 3: Practical Data Preparation

**Objective:** Handle categorical features using One-Hot Encoding and address class imbalance using SMOTE.

## 1. Setup

Import necessary libraries.

In [26]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

## 2. Data Loading

Load the dataset.

In [ ]:
def load_data(file_path):
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data
    """
    # YOUR CODE HERE
    # Load the CSV file using pandas
    
    return pd.read_csv(file_path)

## 3. Categorical Feature Encoding

Implement `encode_categorical_features` using `OneHotEncoder`.

In [27]:
def encode_categorical_features(df, column_to_encode='smoker_status'):
    """
    Encode a categorical column using OneHotEncoder.
    
    Args:
        df: Input DataFrame
        column_to_encode: Name of the categorical column to encode
        
    Returns:
        DataFrame with the categorical column replaced by one-hot encoded columns
    """
    # YOUR CODE HERE
    # 1. Extract the categorical column
    # 2. Apply OneHotEncoder
    # 3. Create new column names
    # 4. Replace the original categorical column with the encoded columns
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    encoded = encoder.fit_transform(df[[column_to_encode]])

    encoded_columns = encoder.get_feature_names_out([column_to_encode])
    encoded_df = pd.DataFrame(encoded, columns=encoded_columns, index=df.index)

    df_encoded = df.drop(columns=[column_to_encode])
    df_encoded = pd.concat([df_encoded, encoded_df], axis=1)

    return df_encoded

## 4. Data Preparation

Implement `prepare_data_part3` to handle the train/test split correctly.

In [28]:
def prepare_data_part3(df, test_size=0.2, random_state=42):
    """
    Prepare data with categorical encoding.
    
    Args:
        df: Input DataFrame
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    # YOUR CODE HERE
    # 1. Encode categorical features using the encode_categorical_features function
    # 2. Select relevant features (including the one-hot encoded ones) and the target
    # 3. Split data into training and testing sets
    # 4. Handle missing values
    df_encoded = encode_categorical_features(df, column_to_encode='smoker_status')

    if 'timestamp' in df_encoded.columns:
        df_encoded = df_encoded.drop(columns=['timestamp'])

    X = df_encoded.drop(columns=['disease_outcome'])
    y = df_encoded['disease_outcome']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    imputer = SimpleImputer(strategy='mean')
    X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(imputer.transform(X_test), columns=X.columns)

    return X_train, X_test, y_train, y_test

## 5. Handling Imbalanced Data

Implement `apply_smote` to oversample the minority class.

In [29]:
def apply_smote(X_train, y_train, random_state=42):
    """
    Apply SMOTE to oversample the minority class.
    
    Args:
        X_train: Training features
        y_train: Training target
        random_state: Random seed for reproducibility
        
    Returns:
        Resampled X_train and y_train with balanced classes
    """
    # YOUR CODE HERE
    # Apply SMOTE to balance the classes
    smote = SMOTE(random_state=random_state)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

    return X_resampled, y_resampled

## 6. Model Training and Evaluation

Train a model on the SMOTE-resampled data and evaluate it.

In [30]:
def train_logistic_regression(X_train, y_train):
    """
    Train a logistic regression model.
    
    Args:
        X_train: Training features
        y_train: Training target
        
    Returns:
        Trained logistic regression model
    """
    # YOUR CODE HERE
    # Initialize and train a LogisticRegression model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    return model

def calculate_evaluation_metrics(model, X_test, y_test):
    """
    Calculate classification evaluation metrics.
    
    Args:
        model: Trained model
        X_test: Test features
        y_test: Test target
        
    Returns:
        Dictionary containing accuracy, precision, recall, f1, auc, and confusion_matrix
    """
    # YOUR CODE HERE
    # 1. Generate predictions
    # 2. Calculate metrics: accuracy, precision, recall, f1, auc
    # 3. Create confusion matrix
    # 4. Return metrics in a dictionary
    
    # Placeholder return - replace with your implementation
    y_pred = model.predict(X_test)
    y_probability = model.predict_proba(X_test)[:, 1]

    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, zero_division=0),
        'recall': recall_score(y_test, y_pred, zero_division=0),
        'f1': f1_score(y_test, y_pred, zero_division=0),
        'auc': roc_auc_score(y_test, y_probability),
        'confusion_matrix': confusion_matrix(y_test, y_pred)
    }

    return metrics

## 7. Save Results

Save the evaluation metrics to a text file.

In [31]:
# YOUR CODE HERE
# 1. Create 'results' directory if it doesn't exist
# 2. Format metrics as strings
# 3. Write metrics to 'results/results_part3.txt'
def save_metrics_part3(metrics, output_path="/workspaces/5-put-a-label-on-it-anagasuri/results/results_part3.txt"):

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(output_path, "w") as f:
        for key, value in metrics.items():
            if key == "confusion_matrix":
                f.write(f"{key}:\n{value.tolist()}\n\n")
            else:
                f.write(f"{key}: {value:.4f}\n")

## 8. Main Execution

Run the complete workflow.

In [37]:
# Main execution
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Prepare data with categorical encoding
    X_train, X_test, y_train, y_test = prepare_data_part3(df)
    
    # 3. Apply SMOTE to balance the training data
    X_train_resampled, y_train_resampled = apply_smote(X_train, y_train)
    
    # 4. Train model on resampled data
    model = train_logistic_regression(X_train_resampled, y_train_resampled)
    
    # 5. Evaluate on original test set
    metrics = calculate_evaluation_metrics(model, X_test, y_test)
    
    # 6. Print metrics
    for metric, value in metrics.items():
        if metric != 'confusion_matrix':
            print(f"{metric}: {value:.4f}")
    
    # 7. Save results
    save_metrics_part3(metrics)
    
    # 8. Load Part 1 results for comparison
    import json
    try:
        with open('results/results_part1.txt', 'r') as f:
            part1_metrics = json.load(f)
        
        # 9. Compare models
        comparison = compare_models(part1_metrics, metrics)
        print("\nModel Comparison (improvement percentages):")
        for metric, improvement in comparison.items():
            print(f"{metric}: {improvement:.2f}%")
    except FileNotFoundError:
        print("Part 1 results not found. Run part1_introduction.ipynb first.")

accuracy: 0.8547
precision: 0.3885
recall: 0.8531
f1: 0.5339
auc: 0.9267

Model Comparison (improvement percentages):
accuracy: -6.77%
precision: -41.26%
recall: 183.72%
f1: 29.12%
auc: 2.01%


## 9. Compare Results

Implement a function to compare model performance between balanced and imbalanced data.

In [36]:
def compare_models(part1_metrics, part3_metrics):
    """
    Calculate percentage improvement between models trained on imbalanced vs. balanced data.
    
    Args:
        part1_metrics: Dictionary containing evaluation metrics from Part 1 (imbalanced)
        part3_metrics: Dictionary containing evaluation metrics from Part 3 (balanced)
        
    Returns:
        Dictionary with metric names as keys and improvement percentages as values
    """
    # YOUR CODE HERE
    # 1. Calculate percentage improvement for each metric
    # 2. Handle metrics where higher is better (most metrics) and where lower is better
    # 3. Return a dictionary with metric names and improvement percentages
    
    def compute_improvement(m1, m3):
        try:
            return ((m3 - m1) / m1) * 100 if m1 != 0 else 0.0
        except:
            return 0.0

    return {
        'accuracy': compute_improvement(part1_metrics.get('accuracy', 0), part3_metrics.get('accuracy', 0)),
        'precision': compute_improvement(part1_metrics.get('precision', 0), part3_metrics.get('precision', 0)),
        'recall': compute_improvement(part1_metrics.get('recall', 0), part3_metrics.get('recall', 0)),
        'f1': compute_improvement(part1_metrics.get('f1', 0), part3_metrics.get('f1', 0)),
        'auc': compute_improvement(part1_metrics.get('auc', 0), part3_metrics.get('auc', 0))
    }